# Melanoma Classification Pre-Trained CNN Model

#### by Arseniy Arsentyev (programpro.ars@gmail.com)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Normalization, Dropout
from keras.applications.vgg19 import VGG19
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
path_to_labels = 'path to the training labels stored in .npy file'
path_to_data = 'path to the training data stored in .npy file'

In [ ]:
training_data = np.load(path_to_data)
training_data = np.reshape(training_data, (1000, 512, 512, -1))
training_labels = np.load(path_to_labels)[:, 1].flatten()
print(training_data.shape)
print(training_labels.shape)

In [ ]:
vgg19 = VGG19(include_top=False, classes=2, input_shape=(512, 512, 3))
vgg19.trainable = False

In [ ]:
model = Sequential()
model.add(Normalization())
model.add(vgg19)
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('pre-trained-cnn-model.h5', monitor='val_loss', save_best_only=True, verbose=1)
callbacks_list = [checkpoint]

training_history = model.fit(training_data, training_labels, validation_split=0.1, epochs=7, callbacks=callbacks_list)

In [ ]:
from keras.models import  load_model
model = load_model('pre-trained-cnn-model.h5')
model.trainable = True
trainable = False
for layer in model.layers:
    if layer.name == 'block5_conv1':
        trainable = True
    layer.trainable = trainable

from keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-6), loss='binary_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint('pre-trained-cnn-model.h5', monitor='val_loss', save_best_only=True, verbose=1)
callbacks_list = [checkpoint]

training_history = model.fit(training_data, training_labels, validation_split=0.25, epochs=7, callbacks=callbacks_list)

plt.plot(training_history.history['loss'])
plt.plot(training_history.history['val_loss'])
plt.show()